<a href="https://colab.research.google.com/github/will-gun/stock-data/blob/master/collectTaiwanStockData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
a = 1
print(a)

1
